In [1]:
from cimbuilder.object_builder.new_house import estimate_num_houses
from cimbuilder.object_builder.new_house import new_house

In [2]:
from cimgraph.databases.fileparsers import XMLFile
from cimgraph.databases import ConnectionParameters
from cimgraph.models import FeederModel, GraphModel
import cimgraph.data_profile.cimhub_2023 as cim 
import cimgraph.utils as utils

In [3]:
params = ConnectionParameters(filename="/home/ande188/CIM-Graph/tests/test_models/ieee13.xml", cim_profile='cimhub_2023', iec61970_301=8)
xml_file = XMLFile(params)
feeder_mrid = "49AD8E07-3BF9-A4E2-CB8F-C3722F837B62"
feeder = cim.Feeder(mRID = feeder_mrid, name = 'ieee13')
feeder_model = FeederModel(connection=xml_file, container=feeder, distributed=False)

In [4]:
def search_by_value(network:GraphModel, class_type:type, attribute:str, value) -> object:
    for uuid in network.graph.get(class_type, {}):
        if getattr(feeder_model.graph[class_type][uuid], attribute) == value:
            return network.graph[class_type][uuid]


In [5]:
load = search_by_value(feeder_model, cim.EnergyConsumer, 'name', 'house')
load.__dict__

{'identifier': UUID('32f02d2b-ee6e-4d3f-8486-1b5caef70204'),
 'mRID': '32F02D2B-EE6E-4D3F-8486-1B5CAEF70204',
 'aliasName': None,
 'description': None,
 'name': 'house',
 'Names': [],
 'AssetDatasheet': None,
 'Assets': [],
 'Controls': [],
 'Location': {"@id": "0c4d68bf-ac73-45c8-b2bd-ff3bfbcd20e1", "@type": "Location"},
 'Measurements': [],
 'PSRType': None,
 'aggregate': None,
 'inService': None,
 'networkAnalysisEnabled': None,
 'normallyInService': None,
 'AdditionalEquipmentContainer': [],
 'EquipmentContainer': {"@id": "49ad8e07-3bf9-a4e2-cb8f-c3722f837b62", "@type": "Feeder"},
 'OperationalLimitSet': [],
 'SubSchedulingArea': None,
 'BaseVoltage': {"@id": "d5404a34-509a-41b1-8caa-44712a11f6c6", "@type": "BaseVoltage"},
 'Terminals': [{"@id": "2128bb42-3e2d-490a-a29d-05549e81f25d", "@type": "Terminal"}],
 'EnergyConnectionProfile': None,
 'customerCount': 1,
 'grounded': True,
 'p': 5000.0,
 'phaseConnection': <PhaseShuntConnectionKind.Y: 'Y'>,
 'q': 3098.7217,
 'EnergyConsumerP

In [6]:
import math
mag_s = math.sqrt(load.p * load.p + load.q * load.q)
mag_s

5882.352945382561

In [8]:
import requests
import json
url = 'https://raw.githubusercontent.com/GRIDAPPSD/CIMHub/refs/heads/feature/SETO/src_python/cimhub/eia_recs/housing_data.json'
try:
    # Send a GET request to the URL
    response = requests.get(url)
    
    # Check if the request was successful
    response.raise_for_status()
    
    # Load the JSON data from the response
    housing_data = response.json()
    
    # # Process the JSON data as needed
    # print("Successfully read JSON data:")
    # print(json.dumps(data, indent=4))  # Pretty print the JSON data
    
    # return data
except requests.exceptions.RequestException as e:
    print(f"HTTP request error: {e}")
except json.JSONDecodeError as e:
    print(f"JSON decoding error: {e}")

In [13]:
estimate_num_houses(housing_data['marine'], mag_s, 10)

10.654021247399436

In [12]:
housing_data.keys()

dict_keys(['marine', 'cold/very cold', 'hot-dry/mixed-dry', 'mixed-humid', 'hot-humid'])

In [14]:
for load in feeder_model.graph[cim.EnergyConsumer].values():
    print(load.p)

1155000.0
160000.0
120000.0
120000.0
170000.0
230000.0
170000.0
485000.0
68000.0
290000.0
170000.0
128000.0
17000.0
66000.0
117000.0
5000.0
